# Key Vault-backed secret scopes

Azure Databricks has two types of secret scopes: Key Vault-backed and Databricks-backed. These secret scopes allow you to store secrets, such as database connection strings, securely. If someone tries to output a secret to a notebook, it is replaced by `[REDACTED]`. This helps prevent someone from viewing the secret or accidentally leaking it when displaying or sharing the notebook.

In this lesson, you will learn how to configure a Key Vault-backed secret scope. You will use Key Vault to store sensitive connection information for Azure SQL Database and Azure Cosmos DB. This notebook walks you through the required steps to create the Key Vault, Azure SQL Database, and Cosmos DB resources, and how to create a Key Vault-backed secret scope.

## Setup

To begin, you will need to create the required resources in the Azure portal. Follow the steps below to create a Key Vault service, Azure SQL Database, and Azure Cosmos DB.

### Azure Key Vault

1. Open <https://portal.azure.com> in a new browser tab.
2. Select the **Create a resource** option on the upper left-hand corner of the Azure portal

    ![Output after Key Vault creation completes](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-keyvault.png)
3. In the Search box, enter **Key Vault**.
4. From the results list, choose **Key Vault**.
5. On the Key Vault section, choose **Create**.
6. On the **Create key vault** section provide the following information:
    - **Name**: A unique name is required. For this quickstart we use **Databricks-Demo**. 
    - **Subscription**: Choose a subscription.
    - Under **Resource Group** select the resource group under which your Azure Databricks workspace is created.
    - In the **Location** pull-down menu, choose the same location as your Azure Databricks workspace.
    - Check the **Pin to dashboard** checkbox.
    - Leave the other options to their defaults.
    
    ![Key Vault creation form](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-keyvault-form.png)
7. After providing the information above, select **Create**.
8. After Key Vault is created, open it and navigate to **Properties**.
9. Copy the **DNS Name** and **Resource ID** properties and paste them to Notebook or some other text application that you can **reference later**.

    ![Key Vault properties](https://databricksdemostore.blob.core.windows.net/images/04-MDW/keyvault-properties.png)
    
### Azure SQL Database

1. In the Azure portal, select the **Create a resource** option on the upper left-hand corner.

    ![Output after SQL Database creation completes](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-sql-database.png)
2. In the Search box, enter **SQL Database**.
3. From the results list, choose **SQL Database**.
4. On the SQL Database section, choose **Create**.
5. On the **Create SQL database** section provide the following information:
    - **Name**: A unique name is required. For this quickstart we use **Databricks-SQL**. 
    - **Subscription**: Choose a subscription.
    - Under **Resource Group** select the resource group under which your Azure Databricks workspace is created.
    - In the **Location** pull-down menu, choose the same location as your Azure Databricks workspace.
    - In the **Select source** pull-down menu, choose **Sample (AdventureWorksLT)**.
    - Under **Server**, either select an existing server or create a new one.
    - Under **Pricing tier**, select **Standard S0: 10 DTUs, 250 GB**.
    - Leave the other options to their defaults.
    
    ![SQL Database creation form](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-sql-database-form.png)
6. After providing the information above, select **Create**.
7. Copy the server's **Host Name**, the SQL **Database Name**, **Username**, and **Password** and paste them to Notebook or some other text application that you can **reference later**.

### Azure Cosmos DB

1. In the Azure portal, select the **Create a resource** option on the upper left-hand corner.

    ![Output after Azure Cosmos DB creation completes](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-cosmos-db.png)
2. In the Search box, enter **Cosmos DB**.
3. From the results list, choose **Cosmos DB**.
4. On the SQL Database section, choose **Create**.
5. On the **Create Cosmos DB** section provide the following information:
    - **Subscription**: Choose a subscription.
    - Under **Resource Group** select the resource group under which your Azure Databricks workspace is created.
    - **Account Name**: A unique name is required. For this quickstart we use **databricks-demo**.
    - In the **API** pull-down menu, choose **Core (SQL)**.
    - In the **Location** pull-down menu, choose the same location as your Azure Databricks workspace.
    - For **Geo-Redundancy**, select **Enable**.
    - For **Multi-region Writes**, select **Enable**.
    - Leave the other options to their defaults.
    
    ![Cosmos DB creation form](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-cosmos-db-form.png)
6. After providing the information above, select **Review + create**.
7. In the review section, select **Create**.
8. After Cosmos DB is created, navigate to it and select **Data Explorer** on the left-hand menu.
9. Select **New Database**, then provide the following options in the form:
    - **Database id**: demos
    - **Provision throughput**: checked
    - **Throughput**: 10000
    
    ![Cosmos DB New Database creation form](https://databricksdemostore.blob.core.windows.net/images/04-MDW/cosmos-db-new-database.png)
10. Select **OK**, then select **New Collection** to add a collection to the database, then provide the following options in the form:
    - **Database id**: select **Use existing** then select **demos**
    - **Collection id**: documents
    - **Partition key**: partitionId
    - **Provision dedicated throughput for this collection**: unchecked
    - **Utilize all of the available database throughput for this collection**: checked
    
    ![Cosmos DB New Collection creation form](https://databricksdemostore.blob.core.windows.net/images/04-MDW/cosmos-db-new-collection.png)
11. Select **OK**.
12. Select **Keys** on the left-hand menu of your new Cosmos DB instance and copy the **URI** and **Primary Key** (or Secondary Key) values and paste them to Notebook or some other text application that you can **reference later**. What you need for connecting to Cosmos DB are:
    - **Endpoint**: Your Cosmos DB url (i.e. https://youraccount.documents.azure.com:443/)
    - **Masterkey**: The primary or secondary key string for you Cosmos DB account
    - **Database**: The name of the database (`demos`)
    - **Collection**: The name of the collection that you wish to query (`documents`)
    
    ![Cosmos DB Keys](https://databricksdemostore.blob.core.windows.net/images/04-MDW/cosmos-db-keys.png)

## Configure Azure Databricks

Now that you have an instance of Key Vault up and running, it is time to let Azure Databricks know how to connect to it.

1. The first step is to open a new web browser tab and navigate to https://<your_azure_databricks_url>#secrets/createScope (for example, https://westus.azuredatabricks.net#secrets/createScope).
2. Enter the name of the secret scope, such as `key-vault-secrets`.
3. Select **Creator** within the Manage Principal drop-down to specify only the creator (which is you) of the secret scope has the MANAGE permission.

  > MANAGE permission allows users to read and write to this secret scope, and, in the case of accounts on the Azure Databricks Premium Plan, to change permissions for the scope.

  > Your account must have the Azure Databricks Premium Plan for you to be able to select Creator. This is the recommended approach: grant MANAGE permission to the Creator when you create the secret scope, and then assign more granular access permissions after you have tested the scope.

4. Enter the **DNS Name** (for example, https://databricks-demo.vault.azure.net/) and **Resource ID** you copied earlier during the Key Vault creation step, for example: `/subscriptions/xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx/resourcegroups/azure-databricks/providers/Microsoft.KeyVault/vaults/Databricks-Demo`. If this is a preconfigured environment, you do not need to complete this step.

  ![Create Secret Scope form](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-secret-scope.png 'Create Secret Scope')
5. Select **Create**.

After a moment, you will see a dialog verifying that the secret scope has been created.

## Create secrets in Key Vault

To create secrets in Key Vault that can be accessed from your new secret scope in Databricks, you need to either use the Azure portal or the Key Vault CLI. For simplicity's sake, we will use the Azure portal:

1. In the Azure portal, navigate to your Key Vault instance.
2. Select **Secrets** in the left-hand menu.
3. Select **+ Generate/Import** in the Secrets toolbar.
4. Provide the following in the Create a secret form, leaving all other values at their defaults:
  - **Name**: sql-username
  - **Value**: the username for your Azure SQL server
  
  ![Create a secret form](https://databricksdemostore.blob.core.windows.net/images/04-MDW/create-secret.png 'Create a secret')
5. Select **Create**.

**Repeat steps 3 - 5** to create the following secrets:

| Secret name | Secret value |
| --- | --- |
| sql-password | the password for your Azure SQL server |
| cosmos-uri | the URI value for your Azure Cosmos DB instance |
| cosmos-key | the key value for your Azure Cosmos DB instance |

When you are finished, you should have the following secrets listed within your Key Vault instance:

![List of Key Vault secrets](https://databricksdemostore.blob.core.windows.net/images/04-MDW/keyvault-secrets.png 'Key Vault secrets')

## Access Key Vault secrets in Azure Databricks

Now that you have created your secrets, it's time to access them from within Azure Databricks.

Run the following cell to retrieve and print a secret.

In [6]:
jdbcUsername = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-username")
print('Username: ' + jdbcUsername)

Notice that the value of `jdbcUsername` when printed out is `[REDACTED]`. This is to prevent your secrets from being exposed.

## Next Steps

Start the next lesson, [SQL Database - Connect using Key Vault]($./09-SQL-Database-Connect-Using-Key-Vault)